In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from utils.adaptacion import *

#otros
import os
import numpy as np
import pandas as pd
import timeit
from sklearn.externals import joblib

tipo = pruebas[1]
dataset_name = datasets[0]
dims = dimensions[dataset_name]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Pruebas con el dataset Amazon

In [3]:
print tipo
print dataset_name
print dims
print data_path

msda
amazon
3000
data


In [4]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
unlabeled = dataset_object.unlabeled
domains = dataset_object.domains

Dataset already splitted


In [5]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X)

print X.shape

print "Todos los datos disponibles obtenidos"

(27677, 3000)
Todos los datos disponibles obtenidos


In [6]:
# se genera un sub-conjunto del dataset
spd = 10000 / len(domains)

X_small = None
for domain in domains:
    temp = dataset_object.get_all_domain_X(domain)
    indexes = np.random.choice(temp.shape[0], spd)
    
    if X_small is None:
        X_small = temp[indexes,:]
    else:
        X_small = np.vstack((X_small, temp[indexes,:]))

#X_small = np.asarray(X_small)
X_small = X
print X_small.shape

(27677, 3000)


```python

saved_paths = [ruta_1,...,ruta_n]
```

In [7]:
t_total_i = timeit.default_timer()

msda_paths = os.path.join(models_path, tipo, "%s_paths.pkl" % dataset_name)

if os.path.exists(msda_paths):
    print "Cargando rutas de modelos adaptados."
    saved_paths = joblib.load(msda_paths)
else:
    print "Creando modelos de adaptacion..."

    noises = [0.3, 0.5, 0.8]
    layers_sizes = [1, 3, 5]

    parametros = {
        'noises': noises,
        'layers': layers_sizes
    }


    saved_paths = msda_theano_pseudo_grid_search(X_small, parametros, models_path, tipo, dataset_name)

    print "Modelos creados.\n"
    
    joblib.dump(saved_paths, msda_paths)
    print "Guardando rutas en %s" % msda_paths
    
t_total_f = timeit.default_timer()

t_total = (t_total_f - t_total_i)/60.

print "Creacion de modelos realizada en %.2fm" % t_total

Creando modelos de adaptacion...
	pr: 0.300 - l: 1
	Entrenando capa  1
	Entrenado en 2.32m

	Guardando modelo en models/msda/amazon_0.pkl
	pr: 0.300 - l: 3
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 6.95m

	Guardando modelo en models/msda/amazon_1.pkl
	pr: 0.300 - l: 5
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenando capa  4
	Entrenando capa  5
	Entrenado en 11.52m

	Guardando modelo en models/msda/amazon_2.pkl
	pr: 0.500 - l: 1
	Entrenando capa  1
	Entrenado en 2.31m

	Guardando modelo en models/msda/amazon_3.pkl
	pr: 0.500 - l: 3
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 6.91m

	Guardando modelo en models/msda/amazon_4.pkl
	pr: 0.500 - l: 5
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenando capa  4
	Entrenando capa  5
	Entrenado en 11.81m

	Guardando modelo en models/msda/amazon_5.pkl
	pr: 0.800 - l: 1
	Entrenando capa  1
	Entrenado en 2.34m

	Guardando modelo en models/msda/ama

```python
domain_msda_paths = [ruta_1,...,ruta_n]
```

In [8]:
best_msda_paths = os.path.join(models_path, tipo, "best_%s_paths.pkl" % dataset_name )
if os.path.exists(best_msda_paths):
    #se cargan las rutas de los modelos guardados
    domain_msda_paths = joblib.load(best_msda_paths)   
else:
    domain_msda_paths = []
    #se obtienen los mejores modelos 
    for domain in domains:
        print "Obteniendo mejor clasificador para %s" % domain
        i = 1

        best_score = 0
        best_model = None
        for msda_model_path in saved_paths:
            
            msda_model = joblib.load(msda_model_path)
            theano_model = msda_model['model']
            pr = msda_model['pr']
            n_layers = msda_model['l']

            print "%d) l=%d, pr=%.3f " % (i, msda_model['l'], msda_model['pr']),

            # se obtienen los datos del dominio
            X_tr = np.asarray(labeled[domain]['X_tr'].todense())
            y_tr = np.asarray(labeled[domain]['y_tr'].todense()).argmax(axis=1)

            # se adaptan los datos
            X_deep = theano_model.predict(X_tr)

            # se obtiene el mejor score con GridSearch
            new_clf = get_best_score(X_deep, y_tr, clasifier='SVC', n_jobs=4)
            new_score = new_clf.best_score_
            print "score: %.4f" % new_score
            # se guarda si es el mejor para el modelo
            if new_score > best_score:
                best_score = new_score
                best_model = msda_model

            i = i+1

        #se guarda el mejor modelo para este dominio
        print "Mejor modelo: l=%d, pr=%.3f" % (best_model['l'], best_model['pr'])
        best_msda_path = os.path.join(models_path, tipo, "%s_%s.pkl" % (dataset_name, domain))
        print "Guardando en %s" % best_msda_path
        joblib.dump(best_model, best_msda_path)
        
        domain_msda_paths.append(best_msda_path)
    
    print "Guardando rutas en %s" % best_msda_paths
    joblib.dump(domain_msda_paths, best_msda_paths)

Obteniendo mejor clasificador para electronics
1) l=1, pr=0.300  score: 0.8281
2) l=3, pr=0.300 

/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with Standar

 score: 0.8294
3) l=5, pr=0.300  score: 0.8169
4) l=1, pr=0.500  score: 0.8287
5) l=3, pr=0.500  score: 0.7944
6) l=5, pr=0.500  score: 0.7325
7) l=1, pr=0.800  score: 0.8269
8) l=3, pr=0.800  score: 0.7269
9) l=5, pr=0.800  score: 0.7100
Mejor modelo: l=3, pr=0.300
Guardando en models/msda/amazon_electronics.pkl
Obteniendo mejor clasificador para dvd
1) l=1, pr=0.300  score: 0.7819
2) l=3, pr=0.300  score: 0.7619
3) l=5, pr=0.300  score: 0.7400
4) l=1, pr=0.500  score: 0.7775
5) l=3, pr=0.500  score: 0.7156
6) l=5, pr=0.500  score: 0.6713
7) l=1, pr=0.800  score: 0.7775
8) l=3, pr=0.800  score: 0.7019
9) l=5, pr=0.800  score: 0.6431
Mejor modelo: l=1, pr=0.300
Guardando en models/msda/amazon_dvd.pkl
Obteniendo mejor clasificador para kitchen
1) l=1, pr=0.300  score: 0.8456
2) l=3, pr=0.300  score: 0.8381
3) l=5, pr=0.300  score: 0.8250
4) l=1, pr=0.500  score: 0.8475
5) l=3, pr=0.500  score: 0.8094
6) l=5, pr=0.500  score: 0.7362
7) l=1, pr=0.800  score: 0.8481
8) l=3, pr=0.800  score

In [9]:
domain_msda_paths

['models/msda/amazon_electronics.pkl',
 'models/msda/amazon_dvd.pkl',
 'models/msda/amazon_kitchen.pkl',
 'models/msda/amazon_books.pkl']

In [10]:
#diccionario para mantener los dominios adaptados
adapted = {}

i = 0
for domain in domains:
    best_msda_path = os.path.join(models_path, tipo, "%s_%s.pkl" % (dataset_name, domain))
    
    if os.path.exists(best_msda_path):
        print "Cargando mejor modelo para %s" % domain
        msda_model = joblib.load(best_msda_path)
        theano_model = msda_model['model']
        
        print "\tAdaptando dominio"
        X_tr = np.asarray(labeled[domain]['X_tr'].todense())

        tr_reps = theano_model.predict(X_tr)

        adapted[domain] = {
            'X_tr': tr_reps
        }
        
        i = i+1
    else:
        print "\tGenerar mejor modelo para %s" % domain
        
print "%d/%d dominios adaptados" % (i, len(domains))

Cargando mejor modelo para electronics
	Adaptando dominio
Cargando mejor modelo para dvd
	Adaptando dominio
Cargando mejor modelo para kitchen
	Adaptando dominio
Cargando mejor modelo para books
	Adaptando dominio
4/4 dominios adaptados


In [11]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
# por cada par posible para adaptar
for src in domains:
    best_msda_path = os.path.join(models_path, tipo, "%s_%s.pkl" % (dataset_name, src))
    msda_adapter = joblib.load(best_msda_path)['model']
    
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de 12" % (i+1)
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            #model_name = "%s_%s_%s.pkl" % (tipo,src, tgt)
            model_name = "indomain_%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, model_name)
            
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            #svc = get_best_score(X_tr, y_tr)
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            
            #transfer error
            #entrenado en dominio src y probado en dominio tgt adaptados
            X_tr_a = adapted[src]['X_tr']
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts_a = msda_adapter.predict(X_ts)
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            print "Entrenando clasificador adaptado."
            svc_a = get_best_score(X_tr_a, y_tr, clasifier='SVC', n_jobs=4)
            
            t_error = 1-svc_a.score(X_ts_a, y_ts)
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['mSDA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            
            i+=1
    
print "\nPruebas completadas"

Tarea 1 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 2 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 3 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 4 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 5 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 6 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 7 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 8 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 9 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 10 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 11 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 12 de 12
Cargando modelo existente.
Entrenando clasificador adaptado.

Pruebas completadas


In [12]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,mSDA,e->d,electronics,dvd,23.00,35.50,12.50
1,mSDA,e->k,electronics,kitchen,17.50,21.25,3.75
2,mSDA,e->b,electronics,books,21.25,33.50,12.25
3,mSDA,d->e,dvd,electronics,20.00,29.25,9.25
4,mSDA,d->k,dvd,kitchen,17.50,23.00,5.50
5,mSDA,d->b,dvd,books,21.25,31.50,10.25
6,mSDA,k->e,kitchen,electronics,20.00,20.00,0.00
7,mSDA,k->d,kitchen,dvd,23.00,32.75,9.75
8,mSDA,k->b,kitchen,books,21.25,33.50,12.25
9,mSDA,b->e,books,electronics,20.00,24.75,4.75


In [13]:
new_scores_path = os.path.join(scores_path, tipo, "%s_%d.csv" % (dataset_name, dims))
print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/msda/amazon_3000.csv
Resultados guardados.
